# RAG System Notebook

This notebook demonstrates the full Retrieval-Augmented Generation (RAG) pipeline.

## Purpose
This notebook integrates the retrieval and generation components to create a complete RAG system. It demonstrates:

1.  **Component Initialization**: Setting up the Retriever and Generator with the Knowledge Base.
2.  **Context Retrieval**: Querying the knowledge base to find relevant code examples and documentation.
3.  **Code Generation**: Using an LLM (e.g., OpenAI, Anthropic) augmented with retrieved context to generate Cirq code from natural language prompts.

## Usage
Run this notebook to test the end-to-end RAG pipeline, from query to generated code.


In [ ]:
import sys
import os
from pathlib import Path

# Add project root to path
project_root = Path("..").resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.cirq_rag_code_assistant.config import get_config, setup_logging
from src.rag.knowledge_base import KnowledgeBase
from src.rag.retriever import Retriever
from src.rag.generator import Generator

# Setup logging
setup_logging()

### Initialize Components
We need to initialize the KnowledgeBase, Retriever, and Generator.

In [ ]:
# Initialize Knowledge Base (this will also initialize the underlying VectorStore)
kb = KnowledgeBase()

# Initialize Retriever
retriever = Retriever(knowledge_base=kb)

# Initialize Generator
# Note: Ensure you have your API keys set in .env or config
try:
    generator = Generator(retriever=retriever)
    print("RAG components initialized successfully.")
except Exception as e:
    print(f"Error initializing Generator: {e}")
    print("Please check your API keys and configuration.")

### Retrieval Demo
Let's see what the retriever finds for a sample query.

In [ ]:
query = "Create a Bell state circuit"

results = retriever.retrieve(query, top_k=3)

print(f"Query: {query}")
print(f"Found {len(results)} results:")
for i, res in enumerate(results):
    print(f"\nResult {i+1} (Score: {res['score']:.4f}):")
    print(f"Description: {res['entry'].get('description', 'N/A')}")
    # print(f"Code: {res['entry'].get('code', 'N/A')[:100]}...") # Truncated code

### Generation Demo
Now let's generate some code using the full pipeline.

In [ ]:
if 'generator' in locals():
    prompt = "Generate a quantum circuit that prepares a Bell state and measures it."
    
    try:
        result = generator.generate(prompt)
        
        print("Generated Code:")
        print("-" * 40)
        print(result['code'])
        print("-" * 40)
        print("Metadata:", result['metadata'])
        
    except Exception as e:
        print(f"Generation failed: {e}")
else:
    print("Generator not initialized.")